In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import concat

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


In [4]:
df = pd.read_csv("2018_contaminantes_promedios_12h.csv")
df = df[['PM10']]
data = series_to_supervised(df, 1)
print(data)

     var1(t-1)    var1(t)
1    52.877419  44.826667
2    44.826667  16.477477
3    16.477477  15.280576
4    15.280576  11.951456
5    11.951456  35.993333
..         ...        ...
723  76.279221  58.105960
724  58.105960  74.960265
725  74.960265  53.875817
726  53.875817  18.517483
727  18.517483  23.318182

[727 rows x 2 columns]


# Aprendizaje por refuerzo

In [8]:
import numpy as np

In [83]:
t = data.sample(1)
t = t.iloc[0,1] - t.iloc[0,0]
print(np.sign(t) == np.random.choice([-1.0, 1.0], 1)[0])

True


In [119]:
# Acciones: bajar, subir
k = 2

# Our action values
Q = [0 for _ in range(k)]

# Número de veces que se toma una acción
N = [0 for _ in range(k)]

# Valor para suavizar greedy
eps = 0.1

def pull(a):
    # Se toma una fila al azar, si se adivina si sube o baja a partir del valor anterior, se gana un punto
    t = data.sample(1)
    # Diferencia entre t y t-1
    t = t.iloc[0,1] - t.iloc[0,0]

    if a == 0:
        a = -1
    # if np.sign(t) == np.random.choice([-1.0, 1.0], 1)[0]:
    if np.sign(t) == a:
        return 1
    else:
        return 0

for i in range(10000):
    if np.random.rand() > eps:
        # Take greedy action most of the time
        a = np.argmax(Q)
    else:
        # Take random action with probability eps
        a = np.random.randint(0, k)
    
    # Collect reward
    reward = pull(a)
    
    # Incremental average
    N[a] += 1
    Q[a] += 1/N[a] * (reward - Q[a])
Q

[0.4881866064710304, 0.4989567809239962]